# <font color=red> Data Exploration
#### This notebook aims to find out useful features 
### <font color=blue> Rules :
#### Since we are a lot contributing to the notebook, please comment your code and use explicit variable names
#### Do not modify the raw data as others would like to work on it
#### Only modify your section (copy-paste others code if needed) so that no clashes occur when pulling/pushing on remote repository

## <font color=green>Hugo

In [1]:
# Specific imports in your section

## <font color=green>Nicolas

In [2]:
# Specific imports in your section

## <font color=green>Mohamed

In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
import nltk
import seaborn as sns

#### Getting the data

In [2]:
train_fname = 'train.csv'
#test_fname = 'test.csv'
X = pd.read_csv(train_fname,header=None)
y = pd.read_csv(train_fname, header=None)[0]
#X_test = pd.read_csv(test_fname, header=None)[0]

In [3]:
X.columns = ['label', 'content']

In [8]:
y.value_counts()

0    3066
1    1349
Name: 0, dtype: int64

In [9]:
index_1 = [index for index in range(X.shape[0]) if y[index]==1]
index_1[0:10]

[6, 8, 11, 12, 15, 18, 23, 27, 28, 29]

#### Some insulting comments

In [49]:
X.content[15] # Wow violent lui ... hahaa

'"Its "faggot" you ignorant fuck plus what you trying to impress us with your girl scout club you dumb shit, fuck you, your FAGGOT ass cousins, and fuck m.o.b. you stupid fuck! What is that a little gang full of wannabes cuz unless its a cartel you aint impressing shit! We butt fuck little bitches like you and your cousins down here and the chop you into pieces cuz your little club ain\'t shit dumb mudafucka! "'

#### Adding some features

In [57]:
X['char_count'] = np.array([len(x) for x in X.content])

In [51]:
X.shape

(4415, 3)

In [58]:
X.head()

,label,content,char_count
0,0,"""Imagine being able say, you know what, no san...",684
1,0,"""""But Jack from Raleigh wasn't done. He came b...",365
2,0,"""the Star box allows you to link your comment ...",175
3,0,"""Cheney,Rush,Nugent.The list is endless.""",41
4,0,"""Obama....I'm blow'n mo smoke up yo arses........",181


## Features

In [4]:
class BadWordCounter():
    def __init__(self):
        with open("my_badlist.txt") as f:
            badwords = [l.strip() for l in f.readlines()]
        self.badwords_ = badwords

    def get_feature_names(self):
        return np.array(['n_words', 'n_chars', 'allcaps', 'max_len',
            'mean_len', '@', '!', 'spaces', 'bad_ratio', 'n_bad',
            'capsratio'])

    def fit(self, documents, y=None):
        return self

    def transform(self, documents):
        ## some handcrafted features!
        n_words = [len(c.split()) for c in documents]
        n_chars = [len(c) for c in documents]
        # number of uppercase words
        allcaps = [np.sum([w.isupper() for w in comment.split()])
               for comment in documents]
        # longest word
        max_word_len = [np.max([len(w) for w in c.split()]) for c in documents]
        # average word length
        mean_word_len = [np.mean([len(w) for w in c.split()])
                                            for c in documents]
        # number of google badwords:
        n_bad = [np.sum([c.lower().count(w) for w in self.badwords_])
                                                for c in documents]
        exclamation = [c.count("!") for c in documents]
        addressing = [c.count("@") for c in documents]
        spaces = [c.count(" ") for c in documents]

        allcaps_ratio = np.array(allcaps) / np.array(n_words, dtype=np.float)
        bad_ratio = np.array(n_bad) / np.array(n_words, dtype=np.float)

        return np.array([n_words, n_chars, allcaps, max_word_len,
            mean_word_len, exclamation, addressing, spaces, bad_ratio, n_bad,
            allcaps_ratio]).T

In [5]:
bad_counter = BadWordCounter()

In [6]:
features = bad_counter.transform(X.content)

In [7]:
features.shape

(4415, 11)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score

In [22]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(features, y, test_size=0.4, random_state=0)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
clf_1 = SVC(kernel='linear', C=1)
clf_2 = LinearRegression()
cross_validation.cross_val_score(clf_1, features, y, cv=5)

In [ ]:
clf = Lin